In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.11 from "d:\HealthMarteAI\env\Scripts\python.exe"
  * The NumPy version is: "1.24.4"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: The specified module could not be found.


In [6]:
print("Loading BMI dataset...")
df = pd.read_csv('bmi.csv')
df.head()

Loading BMI dataset...


,Age,Height,Weight,Bmi,BmiClass
0,61,1.85,109.30,31.935720,Obese Class 1
1,60,1.71,79.02,27.023700,Overweight
2,60,1.55,74.70,31.092612,Obese Class 1
3,60,1.46,35.90,16.841809,Underweight
4,60,1.58,97.10,38.896010,Obese Class 2


In [8]:
# Create BMI classes based on standard BMI categories
def get_bmi_class(bmi):
    if bmi < 18.5:
        return 'Underweight'
    elif bmi < 25:
        return 'Normal Weight'
    elif bmi < 30:
        return 'Overweight'
    elif bmi < 35:
        return 'Obese Class 1'
    elif bmi < 40:
        return 'Obese Class 2'
    else:
        return 'Obese Class 3'

In [10]:
bmi_classes = [get_bmi_class(bmi) for bmi in df['Bmi']]

In [11]:
print("Dataset Overview:")
print(f"Dataset shape: {df.shape}")
print("\nFirst 5 rows:")
print(df.head())

print("\nDataset Info:")
print(df.info())

print("\nBMI Class Distribution:")
print(df['BmiClass'].value_counts())

Dataset Overview:
Dataset shape: (741, 5)

First 5 rows:
   Age  Height  Weight        Bmi       BmiClass
0   61    1.85  109.30  31.935720  Obese Class 1
1   60    1.71   79.02  27.023700     Overweight
2   60    1.55   74.70  31.092612  Obese Class 1
3   60    1.46   35.90  16.841809    Underweight
4   60    1.58   97.10  38.896010  Obese Class 2

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741 entries, 0 to 740
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Age       741 non-null    int64  
 1   Height    741 non-null    float64
 2   Weight    741 non-null    float64
 3   Bmi       741 non-null    float64
 4   BmiClass  741 non-null    object 
dtypes: float64(3), int64(1), object(1)
memory usage: 29.1+ KB
None

BMI Class Distribution:
BmiClass
Normal Weight    342
Overweight       166
Underweight       96
Obese Class 3     62
Obese Class 2     55
Obese Class 1     20
Name: count, dtype: int64


In [12]:
# Prepare the data - CORRECTED APPROACH
# Features: Age, Height, Weight (input features)
# Target: BmiClass (what we want to predict)
X = df[['Age', 'Height', 'Weight']]
y = df['BmiClass']

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"\nFeatures: {list(X.columns)}")
print(f"Target classes: {sorted(y.unique())}")


Feature matrix shape: (741, 3)
Target vector shape: (741,)

Features: ['Age', 'Height', 'Weight']
Target classes: ['Normal Weight', 'Obese Class 1', 'Obese Class 2', 'Obese Class 3', 'Overweight', 'Underweight']


In [13]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining set: {X_train.shape}")
print(f"Test set: {X_test.shape}")


Training set: (592, 3)
Test set: (149, 3)


In [14]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nFeatures scaled successfully!")


Features scaled successfully!


In [15]:
# Train multiple models
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
   'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100)
}
model_results = {}

In [16]:
for name, model in models.items():
    print(f"\n{'-'*40}")
    print(f"Training {name}...")
    
    # Train the model
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    train_pred = model.predict(X_train_scaled)
    test_pred = model.predict(X_test_scaled)
    
    # Calculate accuracies
    train_accuracy = accuracy_score(y_train, train_pred)
    test_accuracy = accuracy_score(y_test, test_pred)
     # Cross-validation score
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5)
    
    model_results[name] = {
        'model': model,
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std()
    }
    
    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Cross-validation Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")


----------------------------------------
Training Logistic Regression...


Training Accuracy: 0.8598
Test Accuracy: 0.8322
Cross-validation Score: 0.8513 (+/- 0.0414)

----------------------------------------
Training Random Forest...
Training Accuracy: 1.0000
Test Accuracy: 0.9262
Cross-validation Score: 0.9206 (+/- 0.0229)


In [17]:
 
# Select best model
best_model_name = max(model_results.keys(), 
                     key=lambda x: model_results[x]['test_accuracy'])
best_model = model_results[best_model_name]['model']

print(f"\n" + "="*60)
print(f"BEST MODEL: {best_model_name}")
print("="*60)


BEST MODEL: Random Forest


In [18]:
# Detailed evaluation of best model
test_pred = best_model.predict(X_test_scaled)
print(f"\nTest Accuracy: {accuracy_score(y_test, test_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, test_pred))


Test Accuracy: 0.9262

Classification Report:
               precision    recall  f1-score   support

Normal Weight       0.97      1.00      0.99        69
Obese Class 1       0.33      0.25      0.29         4
Obese Class 2       0.73      0.73      0.73        11
Obese Class 3       0.85      0.85      0.85        13
   Overweight       0.94      0.94      0.94        33
  Underweight       1.00      0.95      0.97        19

     accuracy                           0.93       149
    macro avg       0.80      0.79      0.79       149
 weighted avg       0.92      0.93      0.92       149



In [ ]:
def predict_bmi_class(age, height, weight, model=best_model, scaler=scaler):
  
    # Create input array
    input_data = np.array([[age, height, weight]])
    
    # Scale the input
    input_scaled = scaler.transform(input_data)
    
    # Predict class
    predicted_class = model.predict(input_scaled)[0]
    
    # Get confidence scores
    if hasattr(model, 'predict_proba'):
        confidence_scores = model.predict_proba(input_scaled)[0]
        class_names = model.classes_
    else:
        confidence_scores = None
        class_names = None
    
    # Calculate actual BMI for comparison
    actual_bmi = weight / (height ** 2)
    
    return predicted_class, actual_bmi, confidence_scores, class_names

In [20]:
# Test the prediction function
print("\nTesting Prediction Function:")
print("-" * 40)

test_cases = [
    (25, 1.75, 70),   # Normal
    (30, 1.60, 45),   # Underweight
    (35, 1.80, 95),   # Overweight
    (40, 1.65, 90),   # Obese
]


Testing Prediction Function:
----------------------------------------
